In [1]:
## for jupyter notebook only
import warnings

warnings.simplefilter("ignore")

from IPython.display import HTML, display

display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)


import json
import os

# Importing libraries
import pandas as pd
from ds.utilities.io import ds_trino

pd.options.display.max_columns = None
pd.options.display.max_rows = 50
pd.options.mode.chained_assignment = None


conn = ds_trino.create_trino_connection()

In [2]:
import numpy as np

In [3]:
from ds.utilities.secrets import secrets
from google.oauth2 import service_account
from google.cloud import bigquery
s = secrets.Secrets()
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"))
client = bigquery.Client(credentials=credentials)

In [4]:
big_query_unica="""


SELECT 
date(DATE_TRUNC(current_date('America/Mexico_City'), week(MONDAY))) as current_week,
extract(isoweek from date(DATE_TRUNC(current_date('America/Mexico_City'), week(MONDAY)))) as current_isoweek,
date(DATE_TRUNC(fecha, week(MONDAY))) as created_week, (EXTRACT(ISOWEEK FROM FECHA)) as isoweek,

org_id,tier,date(Closed_Won_Date__c) as close_won_date,
date(Move_In_Date__c) as move_in_date, date(Start_Date__c) as start_date,
date(Churn_Date__c) as churned_date, date(last_task_date) as last_task_date,
Account_Name_Jira__c,org_name,
Opportunity_ID_18__c,Facility_Name_fixed as facility_name,
Kitchen_Country__c,
MRR_local, storage_ft3,category, subcategory, social_media_presence, CSM, Primary_Churn_Reason__c,
CR_Opp_OB_Status__c, Type, Renewal_Type__c, campaign_name,

avg(avg_up_percentage) as avg_up_percentage,
avg (avg_percentage_total) as avg_percentage_total,
avg(avg_inventory_downtime) as avg_inventory_downtime,
avg(avg_non_inventory_downtime) as avg_non_inventory_downtime,
avg(exchange_rate) as exchange_rate,
sum(gmv_local) as gmv_local,
sum(gmv_usd) as gmv_usd,
sum(orders) as orders,
sum(cancelled_gmv_usd) as cancelled_gmv_usd,
sum(cancelled_gmv_local) as cancelled_gmv_local,
sum(odr_orders) as odr_orders,
sum(Controllable_ord_orders) as Controllable_ord_orders,
sum(Non_controllable_ord_orders) as Non_controllable_ord_orders,
sum(issue_orders) as issue_orders,
sum(cancelled_orders) as cancelled_orders,
avg( avg_discount) as avg_discount,
avg(avg_price_usd) as avg_price_usd,
avg(inv_variation) as inv_variation,
avg(inv_accuracy) as inv_accuracy,
avg(inv_shrink) as inv_shrink,
avg(cast (avg_counting_days as numeric) ) as avg_counting_days,
sum(sm_delivered) as sm_delivered,
sum(total_quantity_expected) as expected_skus,
sum(total_quantity_delivered) as delivered_skus,
sum(sm_late_accepted) as late_sm,
sum(unlinked_items_unconnect_to_ofo +unlinked_categories+unlinked_modifier_groups
+children_mismatch_items+children_mismatch_items+children_mismatch_categories+children_mismatch_modifier_groups
+unpublished_menus+unpublished_items+unpublished_categories+unpublished_modifier_groups+modifier_min_max_mismatches+mismatched_hours+item_name_mismatches+category_name_mismatches+description_mismatches+price_mismatches+availability_mismatch) as sku_issues,
sum(sold_items) as sold_items,
sum(items_wasted) as wasted_items,
avg(quantity_on_hand) as avg_itemns_on_hand,
avg(f_count_availability) as f_count_availability,
sum(lost_gmv_auto86) as lost_gmv_auto86,
sum(lost_gmv_inv) as lost_gmv_inv,
avg(availability_stock_ob) as avg_availability_stock_ob




FROM `css-operations.latam_cr_growth.tabla_unica`

where date(fecha)>='2023-01-01'
and Move_In_Date__c is not null
and Churn_Date__c is null
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
order by 3,33 desc


"""

In [5]:
unica=pd.read_gbq(query=big_query_unica, credentials=credentials,
                 dialect='standard')

In [6]:
unica

,current_week,current_isoweek,created_week,isoweek,org_id,tier,close_won_date,move_in_date,start_date,churned_date,last_task_date,Account_Name_Jira__c,org_name,Opportunity_ID_18__c,facility_name,Kitchen_Country__c,MRR_local,storage_ft3,category,subcategory,social_media_presence,CSM,Primary_Churn_Reason__c,CR_Opp_OB_Status__c,Type,Renewal_Type__c,campaign_name,avg_up_percentage,avg_percentage_total,avg_inventory_downtime,avg_non_inventory_downtime,exchange_rate,gmv_local,gmv_usd,orders,cancelled_gmv_usd,cancelled_gmv_local,odr_orders,Controllable_ord_orders,Non_controllable_ord_orders,issue_orders,cancelled_orders,avg_discount,avg_price_usd,inv_variation,inv_accuracy,inv_shrink,avg_counting_days,sm_delivered,expected_skus,delivered_skus,late_sm,sku_issues,sold_items,wasted_items,avg_itemns_on_hand,f_count_availability,lost_gmv_auto86,lost_gmv_inv,avg_availability_stock_ob
0,2023-09-18,38,2022-12-26,52,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,2020-11-09,2020-12-03,2020-11-16,NaT,2023-06-06,Lowko Sorvetes,Lowko Sorvetes,0065G00000Wsb9LQAR,SAO - Dino,Brazil,1350.00,20.0,CR - Frozen Desserts,CR - Ice Cream,58400.0,Luiza Kravtchenko,None,g - Launched,Legacy Contract,Upsell,Kitchen Lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-09-18,38,2022-12-26,52,dd7d79c0-6d5e-4ee3-8f3d-d19145e028be,Tier 2,2020-11-27,2020-12-15,2020-12-15,NaT,2023-08-02,Brigadeiro com VC,Brigadeiro Com Vc - Pik n' Pak,0065G00000WtiUeQAJ,SAO - Dino,Brazil,350.00,4.0,CR - Desserts,CR - Chocolates,14800.0,Luiza Kravtchenko,None,g - Launched,PiknPak Contract,New,Kitchen Lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-09-18,38,2022-12-26,52,576412a2-e855-487a-a92b-5402b2cd95b4,Tier 3,2020-12-06,2021-01-18,2020-12-20,NaT,NaT,Good Pick,Good Pick - Pik n' Pak,0065G00000XGfhKQAT,SAO - Dino,Brazil,1863.03,20.0,CR - Snacks,CR - Health Snacks,5697.0,Rhayla Freitas Gualberto,None,g - Launched,Legacy Contract,New,Kitchen Lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-09-18,38,2022-12-26,52,dd7d79c0-6d5e-4ee3-8f3d-d19145e028be,Tier 2,2021-01-29,2021-02-12,2022-02-25,NaT,2023-08-02,Brigadeiro com VC,Brigadeiro Com Vc - Pik n' Pak,0065G00000YGe95QAD,SAO - Guararapes (Brooklin),Brazil,368.90,4.0,CR - Desserts,CR - Chocolates,14800.0,Luiza Kravtchenko,None,g - Launched,Legacy Contract,Upsell,Kitchen Lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-09-18,38,2022-12-26,52,dd7d79c0-6d5e-4ee3-8f3d-d19145e028be,Tier 2,2021-01-31,2021-03-01,2021-03-01,NaT,2023-08-02,Brigadeiro com VC,Brigadeiro Com Vc - Pik n' Pak,0065G00000YGx1LQAT,SAO - Rua do Acre,Brazil,350.00,20.0,CR - Desserts,CR - Chocolates,14800.0,Luiza Kravtchenko,None,g - Launched,PiknPak Contract,Upsell,Kitchen Lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24565,2023-09-18,38,2023-09-18,38,61519468-c393-43e8-8936-e6d5d1525e17,Tier 2,2021-01-09,2021-03-27,2021-03-27,NaT,2023-01-26,Unilever Helados/Holanda-Ben&Jerry's,Unilever Mexico,006f400000V1GZYAA3,MEX - Roma,Mexico,5000.00,8.0,CR - Frozen Desserts,CR - Ice Cream,3320.0,Daniela Pulido,None,Completed,Legacy Contract,Upsell,Kitchen Lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.868545,0.0,1.000000,0.000000,8.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24566,2023-09-18,38,2023-09-18,38,be169c79-231d-48cd-81b1-23800c573374,Tie

In [7]:
unica.columns

Index(['current_week', 'current_isoweek', 'created_week', 'isoweek', 'org_id',
       'tier', 'close_won_date', 'move_in_date', 'start_date', 'churned_date',
       'last_task_date', 'Account_Name_Jira__c', 'org_name',
       'Opportunity_ID_18__c', 'facility_name', 'Kitchen_Country__c',
       'MRR_local', 'storage_ft3', 'category', 'subcategory',
       'social_media_presence', 'CSM', 'Primary_Churn_Reason__c',
       'CR_Opp_OB_Status__c', 'Type', 'Renewal_Type__c', 'campaign_name',
       'avg_up_percentage', 'avg_percentage_total', 'avg_inventory_downtime',
       'avg_non_inventory_downtime', 'exchange_rate', 'gmv_local', 'gmv_usd',
       'orders', 'cancelled_gmv_usd', 'cancelled_gmv_local', 'odr_orders',
       'Controllable_ord_orders', 'Non_controllable_ord_orders',
       'issue_orders', 'cancelled_orders', 'avg_discount', 'avg_price_usd',
       'inv_variation', 'inv_accuracy', 'inv_shrink', 'avg_counting_days',
       'sm_delivered', 'expected_skus', 'delivered_skus', 'lat

In [8]:
unica['percentage_odr']=unica['odr_orders']/unica['orders']

In [9]:
unica['percentage_controllable_odr']=unica['Controllable_ord_orders']/unica['orders']

In [10]:
esta_semana=unica['current_isoweek'].value_counts().index.values[0]

In [11]:
esta_semana

38

In [12]:
semana_pasada=esta_semana-1

In [13]:
semana_pasada

37

In [14]:
dos_semanas=esta_semana-2

In [15]:
dos_semanas

36

In [16]:
tres_semanas=esta_semana-3

In [17]:
tres_semanas

35

In [18]:
cuatro_semanas=esta_semana-4

In [19]:
cuatro_semanas

34

In [20]:
cinco_semanas=esta_semana-5

In [21]:
cinco_semanas

33

In [22]:
seis_semanas=esta_semana-6

In [23]:
unica_esta_seama=unica[unica['isoweek']==esta_semana].reset_index(drop=True)

In [24]:
unica_esta_seama.columns

Index(['current_week', 'current_isoweek', 'created_week', 'isoweek', 'org_id',
       'tier', 'close_won_date', 'move_in_date', 'start_date', 'churned_date',
       'last_task_date', 'Account_Name_Jira__c', 'org_name',
       'Opportunity_ID_18__c', 'facility_name', 'Kitchen_Country__c',
       'MRR_local', 'storage_ft3', 'category', 'subcategory',
       'social_media_presence', 'CSM', 'Primary_Churn_Reason__c',
       'CR_Opp_OB_Status__c', 'Type', 'Renewal_Type__c', 'campaign_name',
       'avg_up_percentage', 'avg_percentage_total', 'avg_inventory_downtime',
       'avg_non_inventory_downtime', 'exchange_rate', 'gmv_local', 'gmv_usd',
       'orders', 'cancelled_gmv_usd', 'cancelled_gmv_local', 'odr_orders',
       'Controllable_ord_orders', 'Non_controllable_ord_orders',
       'issue_orders', 'cancelled_orders', 'avg_discount', 'avg_price_usd',
       'inv_variation', 'inv_accuracy', 'inv_shrink', 'avg_counting_days',
       'sm_delivered', 'expected_skus', 'delivered_skus', 'lat

In [25]:
unica_esta_seama[['current_week', 'current_isoweek','org_id',
       'tier','Account_Name_Jira__c', 'Opportunity_ID_18__c',
       'facility_name','Kitchen_Country__c','CSM',
         'orders', 'gmv_usd','f_count_availability','inv_accuracy',
         'inv_shrink' ,'avg_discount','odr_orders',
       'Controllable_ord_orders' ]]

,current_week,current_isoweek,org_id,tier,Account_Name_Jira__c,Opportunity_ID_18__c,facility_name,Kitchen_Country__c,CSM,orders,gmv_usd,f_count_availability,inv_accuracy,inv_shrink,avg_discount,odr_orders,Controllable_ord_orders
0,2023-09-18,38,0f04feb7-3919-48bb-a708-7d13d1738354,Tier 1,Unilever BR - Kibon,0065G00000oshsXQAQ,SAO - Rua do Acre,Brazil,Leonardo Borja,7.0,2974.585216,NaN,1.000000,0.000000,0.066668,0.0,0.0
1,2023-09-18,38,0f04feb7-3919-48bb-a708-7d13d1738354,Tier 1,Unilever BR - Kibon,0066T00001BPdnrQAD,RIO - Camerino,Brazil,Leonardo Borja,13.0,2268.895277,NaN,1.000000,0.000000,NaN,0.0,0.0
2,2023-09-18,38,ed5b3f6d-d4aa-4b8e-95b6-5d978e2979f5,Tier 1,Beloving Sex Shop,0065G00000osDWjQAM,GDL - Av. Mexico,Mexico,Diana Echeverry,1.0,123.723916,NaN,0.984655,0.005115,0.000000,0.0,0.0
3,2023-09-18,38,e8683d22-ea10-4318-b400-1ff3b8b80338,Tier 1,D'doces,0065G00000dO1loQAC,SAO - Dino,Brazil,Luiza Kravtchenko,6.0,48.310062,NaN,0.953798,0.006667,0.232479,0.0,0.0
4,2023-09-18,38,bc367a24-2358-4d24-93d6-75f466ea8d3a,Tier 1,Dom Bebidas,0066T00001BSOxUQAX,SAO - Dino,Brazil,Saulo Vieira,4.0,41.958932,NaN,0.546600,0.425358,NaN,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2023-09-18,38,61519468-c393-43e8-8936-e6d5d1525e17,Tier 2,Unilever Helados/Holanda-Ben&Jerry's,006f400000V1GZYAA3,MEX - Roma,Mexico,Daniela Pulido,NaN,NaN,NaN,1.000000,0.000000,NaN,NaN,NaN
626,2023-09-18,38,be169c79-231d-48cd-81b1-23800c573374,Tier 2,Los Paleteros,006f400000VOd38AAD,SAO - Lapa Clelia,Brazil,Saulo Vieira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,2023-09-18,38,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,006f400000VZDKVAA5,SAO - Guararapes (Brooklin),Brazil,Luiza Kravtchenko,NaN,NaN,NaN,1.000000,0.000000,NaN,NaN,NaN
628,2023-09-18,38,e53bbc96-9c1e-47e0-9f66-df868664c75f,Tier 1,La pantera fresca,006f400000Vx1XQAAZ,MEX - Santa Fe,Mexico,Diana Echeverry,NaN,NaN,NaN,0.979757,0.000000,NaN,NaN,NaN


In [26]:
unica_esta_seama[unica_esta_seama['Opportunity_ID_18__c']=='0065G00000nddLSQAY']

,current_week,current_isoweek,created_week,isoweek,org_id,tier,close_won_date,move_in_date,start_date,churned_date,last_task_date,Account_Name_Jira__c,org_name,Opportunity_ID_18__c,facility_name,Kitchen_Country__c,MRR_local,storage_ft3,category,subcategory,social_media_presence,CSM,Primary_Churn_Reason__c,CR_Opp_OB_Status__c,Type,Renewal_Type__c,campaign_name,avg_up_percentage,avg_percentage_total,avg_inventory_downtime,avg_non_inventory_downtime,exchange_rate,gmv_local,gmv_usd,orders,cancelled_gmv_usd,cancelled_gmv_local,odr_orders,Controllable_ord_orders,Non_controllable_ord_orders,issue_orders,cancelled_orders,avg_discount,avg_price_usd,inv_variation,inv_accuracy,inv_shrink,avg_counting_days,sm_delivered,expected_skus,delivered_skus,late_sm,sku_issues,sold_items,wasted_items,avg_itemns_on_hand,f_count_availability,lost_gmv_auto86,lost_gmv_inv,avg_availability_stock_ob,percentage_odr,percentage_controllable_odr
156,2023-09-18,38,2023-09-18,38,df820c2c-f18e-42c5-bb37-ec1b22566efc,Tier 1,2022-04-18,2022-04-15,2022-04-15,NaT,2023-01-26,Nestle SA,Nespresso,0065G00000nddLSQAY,CLO - Chipichape,Colombia,2640500.0,200.0,CR - Grocery,CR - Coffee & Tea,33600.0,Diana Echeverry,None,None,Legacy Contract,Upsell,no Campaign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
lista_df=[]
lista_semanas=[esta_semana, semana_pasada,dos_semanas, tres_semanas,cuatro_semanas, cinco_semanas, seis_semanas]
mydict={'this_week':esta_semana,'last_week':semana_pasada,'two_weeks':dos_semanas, 
              'three_weeks':tres_semanas,'four_weeks':cuatro_semanas, 'five_weeks':cinco_semanas, 
              'six_weeks':seis_semanas}

columnas=['created_week', 'isoweek','org_id',
       'tier','Account_Name_Jira__c', 'org_name','Opportunity_ID_18__c','social_media_presence',
       'facility_name','Kitchen_Country__c','CSM','churned_date', 'CR_Opp_OB_Status__c','move_in_date',
          'Type', 'Renewal_Type__c', 'campaign_name',
         'orders', 'gmv_usd','f_count_availability','inv_accuracy',
         'inv_shrink' ,'avg_discount','percentage_odr',
       'percentage_controllable_odr']
unico_2=unica[columnas].rename(columns={'f_count_availability':'Inventory_Availability'}).reset_index(drop=True)

for i in lista_semanas:
    nuevo_df=unico_2[unico_2['isoweek']==i].reset_index(drop=True)
    nuevo_df_2=nuevo_df.set_index('Opportunity_ID_18__c')
    if i==esta_semana:
        nuevo_df_3=nuevo_df_2.copy()
    else:
        sufijo=list(mydict.keys())[list(mydict.values()).index(i)]
        nuevo_df_3=nuevo_df_2.add_suffix('_'+sufijo)
    
    lista_df.append(nuevo_df_3)

    


In [28]:
lista_df[1].reset_index()['Opportunity_ID_18__c'].value_counts()

0065G00000nddLSQAY    1
0065G00000dPjxUQAS    1
0066T00001AtOPeQAN    1
0065G00000XGfhKQAT    1
0066T00001BfgBuQAJ    1
                     ..
0065G00000aEWOiQAO    1
0065G00000nbgCjQAI    1
0066T000019DerqQAC    1
0065G00000dP7NjQAK    1
006f400000V1GZYAA3    1
Name: Opportunity_ID_18__c, Length: 630, dtype: int64

In [29]:
semanas=pd.concat(lista_df, axis=1).reset_index()

In [30]:
semanas

,Opportunity_ID_18__c,created_week,isoweek,org_id,tier,Account_Name_Jira__c,org_name,social_media_presence,facility_name,Kitchen_Country__c,CSM,churned_date,CR_Opp_OB_Status__c,move_in_date,Type,Renewal_Type__c,campaign_name,orders,gmv_usd,Inventory_Availability,inv_accuracy,inv_shrink,avg_discount,percentage_odr,percentage_controllable_odr,created_week_last_week,isoweek_last_week,org_id_last_week,tier_last_week,Account_Name_Jira__c_last_week,org_name_last_week,social_media_presence_last_week,facility_name_last_week,Kitchen_Country__c_last_week,CSM_last_week,churned_date_last_week,CR_Opp_OB_Status__c_last_week,move_in_date_last_week,Type_last_week,Renewal_Type__c_last_week,campaign_name_last_week,orders_last_week,gmv_usd_last_week,Inventory_Availability_last_week,inv_accuracy_last_week,inv_shrink_last_week,avg_discount_last_week,percentage_odr_last_week,percentage_controllable_odr_last_week,created_week_two_weeks,isoweek_two_weeks,org_id_two_weeks,tier_two_weeks,Account_Name_Jira__c_two_weeks,org_name_two_weeks,social_media_presence_two_weeks,facility_name_two_weeks,Kitchen_Country__c_two_weeks,CSM_two_weeks,churned_date_two_weeks,CR_Opp_OB_Status__c_two_weeks,move_in_date_two_weeks,Type_two_weeks,Renewal_Type__c_two_weeks,campaign_name_two_weeks,orders_two_weeks,gmv_usd_two_weeks,Inventory_Availability_two_weeks,inv_accuracy_two_weeks,inv_shrink_two_weeks,avg_discount_two_weeks,percentage_odr_two_weeks,percentage_controllable_odr_two_weeks,created_week_three_weeks,isoweek_three_weeks,org_id_three_weeks,tier_three_weeks,Account_Name_Jira__c_three_weeks,org_name_three_weeks,social_media_presence_three_weeks,facility_name_three_weeks,Kitchen_Country__c_three_weeks,CSM_three_weeks,churned_date_three_weeks,CR_Opp_OB_Status__c_three_weeks,move_in_date_three_weeks,Type_three_weeks,Renewal_Type__c_three_weeks,campaign_name_three_weeks,orders_three_weeks,gmv_usd_three_weeks,Inventory_Availability_three_weeks,inv_accuracy_three_weeks,inv_shrink_three_weeks,avg_discount_three_weeks,percentage_odr_three_weeks,percentage_controllable_odr_three_weeks,created_week_four_weeks,isoweek_four_weeks,org_id_four_weeks,tier_four_weeks,Account_Name_Jira__c_four_weeks,org_name_four_weeks,social_media_presence_four_weeks,facility_name_four_weeks,Kitchen_Country__c_four_weeks,CSM_four_weeks,churned_date_four_weeks,CR_Opp_OB_Status__c_four_weeks,move_in_date_four_weeks,Type_four_weeks,Renewal_Type__c_four_weeks,campaign_name_four_weeks,orders_four_weeks,gmv_usd_four_weeks,Inventory_Availability_four_weeks,inv_accuracy_four_weeks,inv_shrink_four_weeks,avg_discount_four_weeks,percentage_odr_four_weeks,percentage_controllable_odr_four_weeks,created_week_five_weeks,isoweek_five_weeks,org_id_five_weeks,tier_five_weeks,Account_Name_Jira__c_five_weeks,org_name_five_weeks,social_media_presence_five_weeks,facility_name_five_weeks,Kitchen_Country__c_five_weeks,CSM_five_weeks,churned_date_five_weeks,CR_Opp_OB_Status__c_five_weeks,move_in_date_five_weeks,Type_five_weeks,Renewal_Type__c_five_weeks,campaign_name_five_weeks,orders_five_weeks,gmv_usd_five_weeks,Inventory_Availability_five_weeks,inv_accuracy_five_weeks,inv_shrink_five_weeks,avg_discount_five_weeks,percentage_odr_five_weeks,percentage_controllable_odr_five_weeks,created_week_six_weeks,isoweek_six_weeks,org_id_six_weeks,tier_six_weeks,Account_Name_Jira__c_six_weeks,org_name_six_weeks,social_media_presence_six_weeks,facility_name_six_weeks,Kitchen_Country__c_six_weeks,CSM_six_weeks,churned_date_six_weeks,CR_Opp_OB_Status__c_six_weeks,move_in_date_six_weeks,Type_six_weeks,Renewal_Type__c_six_weeks,campaign_name_six_weeks,orders_six_weeks,gmv_usd_six_weeks,Inventory_Availability_six_weeks,inv_accuracy_six_weeks,inv_shrink_six_weeks,avg_discount_six_weeks,percentage_odr_six_weeks,percentage_controllable_odr_six_weeks
0,0065G00000oshsXQAQ,2023-09-18,38,0f04feb7-3919-48bb-a708-7d13d1738354,Tier 1,Unilever BR - Kibon,Unilever - Pik n' Pak,315000.0,SAO - Rua do Acre,Brazil,Leonardo Borja,NaT,g - Launched,2022-07-

In [31]:
semanas.columns

Index(['Opportunity_ID_18__c', 'created_week', 'isoweek', 'org_id', 'tier',
       'Account_Name_Jira__c', 'org_name', 'social_media_presence',
       'facility_name', 'Kitchen_Country__c',
       ...
       'Renewal_Type__c_six_weeks', 'campaign_name_six_weeks',
       'orders_six_weeks', 'gmv_usd_six_weeks',
       'Inventory_Availability_six_weeks', 'inv_accuracy_six_weeks',
       'inv_shrink_six_weeks', 'avg_discount_six_weeks',
       'percentage_odr_six_weeks', 'percentage_controllable_odr_six_weeks'],
      dtype='object', length=169)

In [32]:
semanas_2=semanas[['created_week', 'isoweek','org_id',
       'tier','Account_Name_Jira__c','org_name', 'Opportunity_ID_18__c',
       'facility_name','Kitchen_Country__c','social_media_presence',
        'Type', 'Renewal_Type__c', 'campaign_name',           
         'churned_date', 'CR_Opp_OB_Status__c','move_in_date','CSM',
         
         
         
         'orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]]

In [33]:
semanas_2

,created_week,isoweek,org_id,tier,Account_Name_Jira__c,org_name,Opportunity_ID_18__c,facility_name,Kitchen_Country__c,social_media_presence,Type,Renewal_Type__c,campaign_name,churned_date,CR_Opp_OB_Status__c,move_in_date,CSM,orders,orders_last_week,orders_two_weeks,orders_three_weeks,orders_four_weeks,orders_five_weeks,orders_six_weeks,gmv_usd,gmv_usd_last_week,gmv_usd_two_weeks,gmv_usd_three_weeks,gmv_usd_four_weeks,gmv_usd_five_weeks,gmv_usd_six_weeks,Inventory_Availability,Inventory_Availability_last_week,Inventory_Availability_two_weeks,Inventory_Availability_three_weeks,Inventory_Availability_four_weeks,Inventory_Availability_five_weeks,Inventory_Availability_six_weeks,inv_accuracy,inv_accuracy_last_week,inv_accuracy_two_weeks,inv_accuracy_three_weeks,inv_accuracy_four_weeks,inv_accuracy_five_weeks,inv_accuracy_six_weeks,inv_shrink,inv_shrink_last_week,inv_shrink_two_weeks,inv_shrink_three_weeks,inv_shrink_four_weeks,inv_shrink_five_weeks,inv_shrink_six_weeks,avg_discount,avg_discount_last_week,avg_discount_two_weeks,avg_discount_three_weeks,avg_discount_four_weeks,avg_discount_five_weeks,avg_discount_six_weeks,percentage_odr,percentage_odr_last_week,percentage_odr_two_weeks,percentage_odr_three_weeks,percentage_odr_four_weeks,percentage_odr_five_weeks,percentage_odr_six_weeks,percentage_controllable_odr,percentage_controllable_odr_last_week,percentage_controllable_odr_two_weeks,percentage_controllable_odr_three_weeks,percentage_controllable_odr_four_weeks,percentage_controllable_odr_five_weeks,percentage_controllable_odr_six_weeks
0,2023-09-18,38,0f04feb7-3919-48bb-a708-7d13d1738354,Tier 1,Unilever BR - Kibon,Unilever - Pik n' Pak,0065G00000oshsXQAQ,SAO - Rua do Acre,Brazil,315000.0,PiknPak Contract,New,no Campaign,NaT,g - Launched,2022-07-05,Leonardo Borja,7.0,330.0,199.0,206.0,391.0,216.0,174.0,2974.585216,29599.236140,15309.004107,9706.197800,16860.167347,10152.726531,12974.053061,NaN,0.867502,0.803403,0.763642,0.889973,0.916509,0.945373,1.000000,1.000000,1.000000,1.000000,0.997383,0.983862,0.983862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066668,0.107549,0.015183,0.131607,0.128024,0.100614,0.120725,0.00,0.133333,0.065327,0.082524,0.043478,0.027778,0.080460,0.0,0.112121,0.005025,0.033981,0.005115,0.009259,0.017241
1,2023-09-18,38,0f04feb7-3919-48bb-a708-7d13d1738354,Tier 1,Unilever BR - Kibon,Unilever - Pik n' Pak,0066T00001BPdnrQAD,RIO - Camerino,Brazil,315000.0,PiknPak Contract,Upsell,no Campaign,NaT,g - Launched,2023-06-16,Leonardo Borja,13.0,239.0,173.0,202.0,301.0,130.0,134.0,2268.895277,15293.802875,8181.901437,6995.318095,10471.200000,7310.512245,11874.824490,NaN,0.794966,0.761748,0.847990,0.877476,0.933972,0.971027,1.000000,1.000000,0.897039,0.975830,0.885919,0.973854,0.972486,0.000000,0.000000,0.065513,0.003680,0.000000,0.002542,0.014315,NaN,0.079168,0.012281,0.013684,0.046827,0.081492,0.071385,0.00,0.146444,0.075145,0.064356,0.059801,0.046154,0.059701,0.0,0.000000,0.000000,0.014851,0.003322,0.007692,0.000000
2,2023-09-18,38,ed5b3f6d-d4aa-4b8e-95b6-5d978e2979f5,Tier 1,Beloving Sex Shop,Beloving Sex Shop,0065G00000osDWjQAM,GDL - Av. Mexico,Mexico,5245.0,PiknPak Contract,Upsell,no Campaign,NaT,Completed,2022-05-12,Diana Echeverry,1.0,33.0,30.0,31.0,40.0,38.0,30.0,123.723916,1510.379836,1556.194021,1363.021405,2119.942285,1121.427562,946.934040,NaN,0.692390,0.530086,0.398455,0.540666,0.603570,0.523325,0.984655,0.986292,0.983252,0.988487,0.993438,0.992440,0.989726,0.005115,0.004268,0.005089,0.005197,0.003648,0.000532,0.000810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2023-09-18,38,e8683d22-ea10-4318-b400-1ff3b8b80338,Tier 1,D'doces,D Doces - Pik n' Pak,0065G00000dO1loQAC,SAO - Dino,Brazil,20700.0,Legacy Contract,New,no Campaign,NaT,g - Launched,2021-10-01,Luiza Kravtchenko,6.0,449.0,526.0,562.0,511.0,521.0,559.0,48.310062,3386.572895,4411.917864,4512.8

In [34]:
semanas_2 [['orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
            
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]]=semanas_2 [[        'orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]].fillna(0)

In [35]:
semanas_num=semanas_2 [['orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]]

In [36]:
semanas_num.columns

Index(['orders', 'orders_last_week', 'orders_two_weeks', 'orders_three_weeks',
       'orders_four_weeks', 'orders_five_weeks', 'orders_six_weeks', 'gmv_usd',
       'gmv_usd_last_week', 'gmv_usd_two_weeks', 'gmv_usd_three_weeks',
       'gmv_usd_four_weeks', 'gmv_usd_five_weeks', 'gmv_usd_six_weeks',
       'Inventory_Availability', 'Inventory_Availability_last_week',
       'Inventory_Availability_two_weeks',
       'Inventory_Availability_three_weeks',
       'Inventory_Availability_four_weeks',
       'Inventory_Availability_five_weeks', 'Inventory_Availability_six_weeks',
       'inv_accuracy', 'inv_accuracy_last_week', 'inv_accuracy_two_weeks',
       'inv_accuracy_three_weeks', 'inv_accuracy_four_weeks',
       'inv_accuracy_five_weeks', 'inv_accuracy_six_weeks', 'inv_shrink',
       'inv_shrink_last_week', 'inv_shrink_two_weeks',
       'inv_shrink_three_weeks', 'inv_shrink_four_weeks',
       'inv_shrink_five_weeks', 'inv_shrink_six_weeks', 'avg_discount',
       'avg_discount

In [37]:
#semanas_num.dtypes

In [38]:
for i in range(len(semanas_num.columns)):
    #print(semanas_num.columns[i])
    if i==0:
        pass
    else:
        nuevo_nombre='delta_'+semanas_num.columns[i]+'-'+semanas_num.columns[i-1]
        print(nuevo_nombre)
        
        semanas_2[nuevo_nombre]=((semanas_num[semanas_num.columns[i-1]]/semanas_num[semanas_num.columns[i]])-1).replace(np.inf,0).replace(-np.inf,0).fillna(0)

delta_orders_last_week-orders
delta_orders_two_weeks-orders_last_week
delta_orders_three_weeks-orders_two_weeks
delta_orders_four_weeks-orders_three_weeks
delta_orders_five_weeks-orders_four_weeks
delta_orders_six_weeks-orders_five_weeks
delta_gmv_usd-orders_six_weeks
delta_gmv_usd_last_week-gmv_usd
delta_gmv_usd_two_weeks-gmv_usd_last_week
delta_gmv_usd_three_weeks-gmv_usd_two_weeks
delta_gmv_usd_four_weeks-gmv_usd_three_weeks
delta_gmv_usd_five_weeks-gmv_usd_four_weeks
delta_gmv_usd_six_weeks-gmv_usd_five_weeks
delta_Inventory_Availability-gmv_usd_six_weeks
delta_Inventory_Availability_last_week-Inventory_Availability
delta_Inventory_Availability_two_weeks-Inventory_Availability_last_week
delta_Inventory_Availability_three_weeks-Inventory_Availability_two_weeks
delta_Inventory_Availability_four_weeks-Inventory_Availability_three_weeks
delta_Inventory_Availability_five_weeks-Inventory_Availability_four_weeks
delta_Inventory_Availability_six_weeks-Inventory_Availability_five_weeks
delt

In [39]:
semanas_exp=semanas_2[['created_week', 'isoweek','org_id',
       'tier','Account_Name_Jira__c','org_name', 'Opportunity_ID_18__c',
       'facility_name','Kitchen_Country__c','social_media_presence',
        'Type', 'Renewal_Type__c', 'campaign_name',               
         'churned_date', 'CR_Opp_OB_Status__c','move_in_date','CSM',
         
         
         
         'orders','orders_last_week','delta_orders_last_week-orders','orders_two_weeks','delta_orders_two_weeks-orders_last_week','orders_three_weeks','delta_orders_three_weeks-orders_two_weeks',
        'orders_four_weeks','delta_orders_four_weeks-orders_three_weeks','orders_five_weeks','delta_orders_five_weeks-orders_four_weeks','orders_six_weeks','delta_orders_six_weeks-orders_five_weeks',
                       
                       
         'gmv_usd', 'gmv_usd_last_week','delta_gmv_usd_last_week-gmv_usd','gmv_usd_two_weeks','delta_gmv_usd_two_weeks-gmv_usd_last_week','gmv_usd_three_weeks','delta_gmv_usd_three_weeks-gmv_usd_two_weeks',
        'gmv_usd_four_weeks', 'delta_gmv_usd_four_weeks-gmv_usd_three_weeks','gmv_usd_five_weeks','delta_gmv_usd_five_weeks-gmv_usd_four_weeks','gmv_usd_six_weeks','delta_gmv_usd_six_weeks-gmv_usd_five_weeks',
                       
                       
                       
         'Inventory_Availability','Inventory_Availability_last_week','delta_Inventory_Availability_last_week-Inventory_Availability','Inventory_Availability_two_weeks','delta_Inventory_Availability_two_weeks-Inventory_Availability_last_week',
        'Inventory_Availability_three_weeks','delta_Inventory_Availability_three_weeks-Inventory_Availability_two_weeks','Inventory_Availability_four_weeks','delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks',
        'Inventory_Availability_five_weeks','delta_Inventory_Availability_five_weeks-Inventory_Availability_four_weeks','Inventory_Availability_six_weeks','delta_Inventory_Availability_six_weeks-Inventory_Availability_five_weeks',
         
                       
                       
        'inv_accuracy','inv_accuracy_last_week','delta_inv_accuracy_last_week-inv_accuracy','inv_accuracy_two_weeks','delta_inv_accuracy_two_weeks-inv_accuracy_last_week','inv_accuracy_three_weeks','delta_inv_accuracy_three_weeks-inv_accuracy_two_weeks',
        'inv_accuracy_four_weeks','delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks','inv_accuracy_five_weeks','delta_inv_accuracy_five_weeks-inv_accuracy_four_weeks','inv_accuracy_six_weeks','delta_inv_accuracy_six_weeks-inv_accuracy_five_weeks',
                       
                       
                       
         'inv_shrink' , 'inv_shrink_last_week','delta_inv_shrink_last_week-inv_shrink','inv_shrink_two_weeks', 'inv_shrink_three_weeks','delta_inv_shrink_three_weeks-inv_shrink_two_weeks',
        'inv_shrink_four_weeks','delta_Inventory_Availability_four_weeks-Inventory_Availability_three_weeks','inv_shrink_five_weeks','delta_inv_shrink_five_weeks-inv_shrink_four_weeks','inv_shrink_six_weeks','delta_inv_shrink_six_weeks-inv_shrink_five_weeks',
                       
                       
                       
         'avg_discount','avg_discount_last_week','delta_avg_discount_last_week-avg_discount','avg_discount_two_weeks','delta_avg_discount_two_weeks-avg_discount_last_week','avg_discount_three_weeks','delta_avg_discount_three_weeks-avg_discount_two_weeks',
        'avg_discount_four_weeks','delta_avg_discount_four_weeks-avg_discount_three_weeks','avg_discount_five_weeks', 'delta_avg_discount_five_weeks-avg_discount_four_weeks','avg_discount_six_weeks','delta_avg_discount_six_weeks-avg_discount_five_weeks',
                       
                       
         'percentage_odr','percentage_odr_last_week','delta_percentage_odr_last_week-percentage_odr','percentage_odr_two_weeks', 'delta_percentage_odr_two_weeks-percentage_odr_last_week',
           'percentage_odr_three_weeks','delta_percentage_odr_three_weeks-percentage_odr_two_weeks','percentage_odr_four_weeks','delta_percentage_odr_four_weeks-percentage_odr_three_weeks',
        'percentage_odr_five_weeks','delta_percentage_odr_five_weeks-percentage_odr_four_weeks','percentage_odr_six_weeks','delta_percentage_controllable_odr-percentage_odr_six_weeks',
                       
         'percentage_controllable_odr','percentage_controllable_odr_last_week','delta_percentage_controllable_odr_last_week-percentage_controllable_odr',
        'percentage_controllable_odr_two_weeks','delta_percentage_controllable_odr_last_week-percentage_controllable_odr','percentage_controllable_odr_three_weeks','delta_percentage_controllable_odr_three_weeks-percentage_controllable_odr_two_weeks',
        'percentage_controllable_odr_four_weeks','delta_percentage_controllable_odr_four_weeks-percentage_controllable_odr_three_weeks','percentage_controllable_odr_five_weeks','delta_percentage_controllable_odr_five_weeks-percentage_controllable_odr_four_weeks',
            'percentage_controllable_odr_six_weeks','delta_percentage_controllable_odr_six_weeks-percentage_controllable_odr_five_weeks'
        ]]

In [40]:
semanas_exp

,created_week,isoweek,org_id,tier,Account_Name_Jira__c,org_name,Opportunity_ID_18__c,facility_name,Kitchen_Country__c,social_media_presence,Type,Renewal_Type__c,campaign_name,churned_date,CR_Opp_OB_Status__c,move_in_date,CSM,orders,orders_last_week,delta_orders_last_week-orders,orders_two_weeks,delta_orders_two_weeks-orders_last_week,orders_three_weeks,delta_orders_three_weeks-orders_two_weeks,orders_four_weeks,delta_orders_four_weeks-orders_three_weeks,orders_five_weeks,delta_orders_five_weeks-orders_four_weeks,orders_six_weeks,delta_orders_six_weeks-orders_five_weeks,gmv_usd,gmv_usd_last_week,delta_gmv_usd_last_week-gmv_usd,gmv_usd_two_weeks,delta_gmv_usd_two_weeks-gmv_usd_last_week,gmv_usd_three_weeks,delta_gmv_usd_three_weeks-gmv_usd_two_weeks,gmv_usd_four_weeks,delta_gmv_usd_four_weeks-gmv_usd_three_weeks,gmv_usd_five_weeks,delta_gmv_usd_five_weeks-gmv_usd_four_weeks,gmv_usd_six_weeks,delta_gmv_usd_six_weeks-gmv_usd_five_weeks,Inventory_Availability,Inventory_Availability_last_week,delta_Inventory_Availability_last_week-Inventory_Availability,Inventory_Availability_two_weeks,delta_Inventory_Availability_two_weeks-Inventory_Availability_last_week,Inventory_Availability_three_weeks,delta_Inventory_Availability_three_weeks-Inventory_Availability_two_weeks,Inventory_Availability_four_weeks,delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks,Inventory_Availability_five_weeks,delta_Inventory_Availability_five_weeks-Inventory_Availability_four_weeks,Inventory_Availability_six_weeks,delta_Inventory_Availability_six_weeks-Inventory_Availability_five_weeks,inv_accuracy,inv_accuracy_last_week,delta_inv_accuracy_last_week-inv_accuracy,inv_accuracy_two_weeks,delta_inv_accuracy_two_weeks-inv_accuracy_last_week,inv_accuracy_three_weeks,delta_inv_accuracy_three_weeks-inv_accuracy_two_weeks,inv_accuracy_four_weeks,delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks,inv_accuracy_five_weeks,delta_inv_accuracy_five_weeks-inv_accuracy_four_weeks,inv_accuracy_six_weeks,delta_inv_accuracy_six_weeks-inv_accuracy_five_weeks,inv_shrink,inv_shrink_last_week,delta_inv_shrink_last_week-inv_shrink,inv_shrink_two_weeks,inv_shrink_three_weeks,delta_inv_shrink_three_weeks-inv_shrink_two_weeks,inv_shrink_four_weeks,delta_Inventory_Availability_four_weeks-Inventory_Availability_three_weeks,inv_shrink_five_weeks,delta_inv_shrink_five_weeks-inv_shrink_four_weeks,inv_shrink_six_weeks,delta_inv_shrink_six_weeks-inv_shrink_five_weeks,avg_discount,avg_discount_last_week,delta_avg_discount_last_week-avg_discount,avg_discount_two_weeks,delta_avg_discount_two_weeks-avg_discount_last_week,avg_discount_three_weeks,delta_avg_discount_three_weeks-avg_discount_two_weeks,avg_discount_four_weeks,delta_avg_discount_four_weeks-avg_discount_three_weeks,avg_discount_five_weeks,delta_avg_discount_five_weeks-avg_discount_four_weeks,avg_discount_six_weeks,delta_avg_discount_six_weeks-avg_discount_five_weeks,percentage_odr,percentage_odr_last_week,delta_percentage_odr_last_week-percentage_odr,percentage_odr_two_weeks,delta_percentage_odr_two_weeks-percentage_odr_last_week,percentage_odr_three_weeks,delta_percentage_odr_three_weeks-percentage_odr_two_weeks,percentage_odr_four_weeks,delta_percentage_odr_four_weeks-percentage_odr_three_weeks,percentage_odr_five_weeks,delta_percentage_odr_five_weeks-percentage_odr_four_weeks,percentage_odr_six_weeks,delta_percentage_controllable_odr-percentage_odr_six_weeks,percentage_controllable_odr,percentage_controllable_odr_last_week,delta_percentage_controllable_odr_last_week-percentage_controllable_odr,percentage_controllable_odr_two_weeks,delta_percentage_controllable_odr_last_week-percentage_controllable_odr,percentage_controllable_odr_three_weeks,delta_percentage_controllable_odr_three_weeks-percentage_controllable_odr_two_weeks,percentage_controllable_odr_four_weeks,delta_percentage_controllable_odr_four_weeks-percentage_controllable_odr_three_weeks,percentage_controllable_odr_five_weeks,delta_percentage_controllable_odr_five_weeks-p

In [41]:
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#gc = pygsheets.authorize(custom_credentials=credentials)

In [42]:
import pygsheets
# Everything ready to upload to GSheets
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


In [43]:
# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1zir2UeDnoOC12k9pfu913s4BbkX2Qs2MTbHjlta0enM')

# select sheet

ws = ss.worksheet('title','Op data')

print('Clearing Range ...')
ws.clear(start='A', end='DT')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(semanas_exp, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [44]:
big_query_unica_cuenta="""


SELECT 
date(DATE_TRUNC(current_date('America/Mexico_City'), week(MONDAY))) as current_week,
extract(isoweek from date(DATE_TRUNC(current_date('America/Mexico_City'), week(MONDAY)))) as current_isoweek,
date(DATE_TRUNC(fecha, week(MONDAY))) as created_week, (EXTRACT(ISOWEEK FROM FECHA)) as isoweek,

org_id,tier, date(last_task_date) as last_task_date,
Account_Name_Jira__c,accountID, org_name,
Kitchen_Country__c,category, subcategory, social_media_presence, CSM, 

avg(avg_up_percentage) as avg_up_percentage,
avg (avg_percentage_total) as avg_percentage_total,
avg(avg_inventory_downtime) as avg_inventory_downtime,
avg(avg_non_inventory_downtime) as avg_non_inventory_downtime,
avg(exchange_rate) as exchange_rate,
sum(gmv_local) as gmv_local,
sum(gmv_usd) as gmv_usd,
sum(orders) as orders,
sum(cancelled_gmv_usd) as cancelled_gmv_usd,
sum(cancelled_gmv_local) as cancelled_gmv_local,
sum(odr_orders) as odr_orders,
sum(Controllable_ord_orders) as Controllable_ord_orders,
sum(Non_controllable_ord_orders) as Non_controllable_ord_orders,
sum(issue_orders) as issue_orders,
sum(cancelled_orders) as cancelled_orders,
avg( avg_discount) as avg_discount,
avg(avg_price_usd) as avg_price_usd,
avg(inv_variation) as inv_variation,
avg(inv_accuracy) as inv_accuracy,
avg(inv_shrink) as inv_shrink,
avg(cast (avg_counting_days as numeric) ) as avg_counting_days,
sum(sm_delivered) as sm_delivered,
sum(total_quantity_expected) as expected_skus,
sum(total_quantity_delivered) as delivered_skus,
sum(sm_late_accepted) as late_sm,
sum(unlinked_items_unconnect_to_ofo +unlinked_categories+unlinked_modifier_groups
+children_mismatch_items+children_mismatch_items+children_mismatch_categories+children_mismatch_modifier_groups
+unpublished_menus+unpublished_items+unpublished_categories+unpublished_modifier_groups+modifier_min_max_mismatches+mismatched_hours+item_name_mismatches+category_name_mismatches+description_mismatches+price_mismatches+availability_mismatch) as sku_issues,
sum(sold_items) as sold_items,
sum(items_wasted) as wasted_items,
avg(quantity_on_hand) as avg_itemns_on_hand,
avg(f_count_availability) as f_count_availability,
sum(lost_gmv_auto86) as lost_gmv_auto86,
sum(lost_gmv_inv) as lost_gmv_inv,
avg(availability_stock_ob) as avg_availability_stock_ob




FROM `css-operations.latam_cr_growth.tabla_unica`

where date(fecha)>='2023-01-01'
and Move_In_Date__c is not null
and Churn_Date__c is null
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
order by 3 desc



"""

In [45]:
unica_cuenta=pd.read_gbq(query=big_query_unica_cuenta, credentials=credentials,
                 dialect='standard')

In [46]:
unica_cuenta

,current_week,current_isoweek,created_week,isoweek,org_id,tier,last_task_date,Account_Name_Jira__c,accountID,org_name,Kitchen_Country__c,category,subcategory,social_media_presence,CSM,avg_up_percentage,avg_percentage_total,avg_inventory_downtime,avg_non_inventory_downtime,exchange_rate,gmv_local,gmv_usd,orders,cancelled_gmv_usd,cancelled_gmv_local,odr_orders,Controllable_ord_orders,Non_controllable_ord_orders,issue_orders,cancelled_orders,avg_discount,avg_price_usd,inv_variation,inv_accuracy,inv_shrink,avg_counting_days,sm_delivered,expected_skus,delivered_skus,late_sm,sku_issues,sold_items,wasted_items,avg_itemns_on_hand,f_count_availability,lost_gmv_auto86,lost_gmv_inv,avg_availability_stock_ob
0,2023-09-18,38,2023-09-18,38,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,2023-06-06,Lowko Sorvetes,001f400001MFS1fAAH,Lowko Sorvetes,Brazil,CR - Frozen Desserts,CR - Ice Cream,58400.0,Luiza Kravtchenko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.625954,-2.000,0.955827,0.011187,8.451549370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-09-18,38,2023-09-18,38,dd7d79c0-6d5e-4ee3-8f3d-d19145e028be,Tier 2,2023-08-02,Brigadeiro com VC,0015G00001VcL4EQAV,Brigadeiro Com Vc - Pik n' Pak,Brazil,CR - Desserts,CR - Chocolates,14800.0,Luiza Kravtchenko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.821355,0.000,1.000000,0.000000,3.571428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-09-18,38,2023-09-18,38,576412a2-e855-487a-a92b-5402b2cd95b4,Tier 3,NaT,Good Pick,0015G00001VfcFOQAZ,Good Pick - Pik n' Pak,Brazil,CR - Snacks,CR - Health Snacks,5697.0,Rhayla Freitas Gualberto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.650924,0.000,1.000000,0.000000,1.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-09-18,38,2023-09-18,38,c0b76b32-019e-49b5-9c27-b1cc2d042770,Tier 3,NaT,Mezcal Amores,0015G00001a90ccQAA,Mezcal Amores,Mexico,CR - Alcoholic Beverages,CR - Spirits,119000.0,Diana Echeverry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.917840,0.000,1.000000,0.000000,6.428571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-09-18,38,2023-09-18,38,1ee553b1-0874-4bc4-a5ce-407a2b943eef,Tier 1,2023-08-24,Pudim Box,001f400001Rk22DAAR,Pudim Box,Brazil,CR - Baked Goods,None,4047.0,Luiza Kravtchenko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.036797,-2.375,0.895323,0.074117,3.817439711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11500,2023-09-18,38,2022-12-26,52,09cd6543-9695-4f8f-84af-4886c85c0243,Tier 3,NaT,CONFITARIA,0016T000035DmzNQAS,Confitaria - Pik n' Pak,Brazil,CR - Baked Goods,CR - Cakes,332.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11501,2023-09-18,38,2022-12-26,52,b376d418-c2ae-4ac8-b89d-a453cfd7d7ee,Tier 3,NaT,CBD Life,0016T00002xKxPQQA0,CBD Life,Mexico,CR - CBD Products,None,19100.0,Daniela Pulido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11502,2023-09-18,38,2022-12-26,52,08a70ed1-3547-4dde-ac11-daf15b128735,Tier 3,NaT,Melhor Pavê do Mundo,0016T000035E0I2QAK,Melhor Pavê do Mundo - Pik n' Pak,Brazil,CR - Baked Goods,CR - Cakes,1442.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11503,2023-09-18,38,2022-12-26,52,60132096-04f5-4c25-b470-554e3bfaf29a,Tier 2,NaT,Parman Incensos,0016T000035GDaAQAW,Parman Incensos - Pik n' Pak,Brazil,CR - Not Listed,None,5000.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
unica_cuenta.columns

Index(['current_week', 'current_isoweek', 'created_week', 'isoweek', 'org_id',
       'tier', 'last_task_date', 'Account_Name_Jira__c', 'accountID',
       'org_name', 'Kitchen_Country__c', 'category', 'subcategory',
       'social_media_presence', 'CSM', 'avg_up_percentage',
       'avg_percentage_total', 'avg_inventory_downtime',
       'avg_non_inventory_downtime', 'exchange_rate', 'gmv_local', 'gmv_usd',
       'orders', 'cancelled_gmv_usd', 'cancelled_gmv_local', 'odr_orders',
       'Controllable_ord_orders', 'Non_controllable_ord_orders',
       'issue_orders', 'cancelled_orders', 'avg_discount', 'avg_price_usd',
       'inv_variation', 'inv_accuracy', 'inv_shrink', 'avg_counting_days',
       'sm_delivered', 'expected_skus', 'delivered_skus', 'late_sm',
       'sku_issues', 'sold_items', 'wasted_items', 'avg_itemns_on_hand',
       'f_count_availability', 'lost_gmv_auto86', 'lost_gmv_inv',
       'avg_availability_stock_ob'],
      dtype='object')

In [48]:
unica_cuenta['percentage_odr']=unica_cuenta['odr_orders']/unica_cuenta['orders']

In [49]:
unica_cuenta['Controllable_ord_orders'].value_counts(dropna=False)

NaN     6146
0.0     4017
1.0      805
2.0      247
3.0      123
4.0       55
5.0       35
6.0       26
7.0       15
8.0        8
9.0        7
10.0       5
11.0       4
13.0       4
15.0       2
12.0       2
37.0       1
29.0       1
14.0       1
19.0       1
Name: Controllable_ord_orders, dtype: int64

In [50]:
unica_cuenta['percentage_controllable_odr']=unica_cuenta['Controllable_ord_orders']/unica_cuenta['orders']

In [51]:
#unica_cuenta[unica_cuenta['isoweek']==24]['percentage_controllable_odr'].value_counts()

In [52]:
unica_cuenta.columns

Index(['current_week', 'current_isoweek', 'created_week', 'isoweek', 'org_id',
       'tier', 'last_task_date', 'Account_Name_Jira__c', 'accountID',
       'org_name', 'Kitchen_Country__c', 'category', 'subcategory',
       'social_media_presence', 'CSM', 'avg_up_percentage',
       'avg_percentage_total', 'avg_inventory_downtime',
       'avg_non_inventory_downtime', 'exchange_rate', 'gmv_local', 'gmv_usd',
       'orders', 'cancelled_gmv_usd', 'cancelled_gmv_local', 'odr_orders',
       'Controllable_ord_orders', 'Non_controllable_ord_orders',
       'issue_orders', 'cancelled_orders', 'avg_discount', 'avg_price_usd',
       'inv_variation', 'inv_accuracy', 'inv_shrink', 'avg_counting_days',
       'sm_delivered', 'expected_skus', 'delivered_skus', 'late_sm',
       'sku_issues', 'sold_items', 'wasted_items', 'avg_itemns_on_hand',
       'f_count_availability', 'lost_gmv_auto86', 'lost_gmv_inv',
       'avg_availability_stock_ob', 'percentage_odr',
       'percentage_controllable_odr

In [53]:
unica_cuenta_esta_seama=unica_cuenta[unica_cuenta['isoweek']==esta_semana].reset_index(drop=True)

In [54]:
unica_cuenta_esta_seama

,current_week,current_isoweek,created_week,isoweek,org_id,tier,last_task_date,Account_Name_Jira__c,accountID,org_name,Kitchen_Country__c,category,subcategory,social_media_presence,CSM,avg_up_percentage,avg_percentage_total,avg_inventory_downtime,avg_non_inventory_downtime,exchange_rate,gmv_local,gmv_usd,orders,cancelled_gmv_usd,cancelled_gmv_local,odr_orders,Controllable_ord_orders,Non_controllable_ord_orders,issue_orders,cancelled_orders,avg_discount,avg_price_usd,inv_variation,inv_accuracy,inv_shrink,avg_counting_days,sm_delivered,expected_skus,delivered_skus,late_sm,sku_issues,sold_items,wasted_items,avg_itemns_on_hand,f_count_availability,lost_gmv_auto86,lost_gmv_inv,avg_availability_stock_ob,percentage_odr,percentage_controllable_odr
0,2023-09-18,38,2023-09-18,38,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,2023-06-06,Lowko Sorvetes,001f400001MFS1fAAH,Lowko Sorvetes,Brazil,CR - Frozen Desserts,CR - Ice Cream,58400.0,Luiza Kravtchenko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.625954,-2.000,0.955827,0.011187,8.451549370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-09-18,38,2023-09-18,38,dd7d79c0-6d5e-4ee3-8f3d-d19145e028be,Tier 2,2023-08-02,Brigadeiro com VC,0015G00001VcL4EQAV,Brigadeiro Com Vc - Pik n' Pak,Brazil,CR - Desserts,CR - Chocolates,14800.0,Luiza Kravtchenko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.821355,0.000,1.000000,0.000000,3.571428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-09-18,38,2023-09-18,38,576412a2-e855-487a-a92b-5402b2cd95b4,Tier 3,NaT,Good Pick,0015G00001VfcFOQAZ,Good Pick - Pik n' Pak,Brazil,CR - Snacks,CR - Health Snacks,5697.0,Rhayla Freitas Gualberto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.650924,0.000,1.000000,0.000000,1.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-09-18,38,2023-09-18,38,c0b76b32-019e-49b5-9c27-b1cc2d042770,Tier 3,NaT,Mezcal Amores,0015G00001a90ccQAA,Mezcal Amores,Mexico,CR - Alcoholic Beverages,CR - Spirits,119000.0,Diana Echeverry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.917840,0.000,1.000000,0.000000,6.428571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-09-18,38,2023-09-18,38,1ee553b1-0874-4bc4-a5ce-407a2b943eef,Tier 1,2023-08-24,Pudim Box,001f400001Rk22DAAR,Pudim Box,Brazil,CR - Baked Goods,None,4047.0,Luiza Kravtchenko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.036797,-2.375,0.895323,0.074117,3.817439711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,2023-09-18,38,2023-09-18,38,9629c7c5-ebc4-4747-8908-f383633859c9,Tier 2,2023-05-10,American Brownie,001f400001OdutJAAR,American Brownies,Brazil,CR - Baked Goods,CR - Brownies,10700.0,Rhayla Freitas Gualberto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1.000000,0.000000,2.545454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291,2023-09-18,38,2023-09-18,38,0f04feb7-3919-48bb-a708-7d13d1738354,Tier 1,2023-02-13,Unilever BR - Kibon,0015G000024533yQAA,Unilever - Pik n' Pak,Brazil,CR - Frozen Desserts,CR - Ice Cream,315000.0,Leonardo Borja,NaN,NaN,NaN,NaN,4.87,25535.75,5243.480493,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066668,NaN,0.000,1.000000,0.000000,1.956131605,NaN,NaN,NaN,NaN,NaN,173.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
292,2023-09-18,38,2023-09-18,38,9443fca3-d5f6-45ac-9c1d-5df53cb2e76b,Tier 3,2023-05-26,aO² Brownie,0016T00002yO5xsQAC,aO² Brownie - Pik n’ Pak,Brazil,CR - Baked Goods,CR - Brownies,10400.0,Rhayla Freitas Gualberto,NaN,NaN,NaN,NaN,4.87,27.56,5.659138,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.000,1.000000,0.000000,5.375000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
293,2023-09-18,38,2023-09-18,38,96249864-893b-465c-a343-56b7b4054d71,Tier 3

In [55]:
unica_cuenta_esta_seama['Account_Name_Jira__c'].value_counts()

Palanda                  2
Lowko Sorvetes           1
NaturALL Fit             1
Artisano Gelato          1
Chilaquiles Mañaneros    1
                        ..
The Healthy Bakery       1
Padalova Prado           1
Aylu Cookies             1
Calábria Carnes          1
Fazedô de Doce           1
Name: Account_Name_Jira__c, Length: 294, dtype: int64

In [56]:
unica_cuenta_esta_seama[unica_cuenta_esta_seama['Account_Name_Jira__c']=='Gelati Artigianale']

,current_week,current_isoweek,created_week,isoweek,org_id,tier,last_task_date,Account_Name_Jira__c,accountID,org_name,Kitchen_Country__c,category,subcategory,social_media_presence,CSM,avg_up_percentage,avg_percentage_total,avg_inventory_downtime,avg_non_inventory_downtime,exchange_rate,gmv_local,gmv_usd,orders,cancelled_gmv_usd,cancelled_gmv_local,odr_orders,Controllable_ord_orders,Non_controllable_ord_orders,issue_orders,cancelled_orders,avg_discount,avg_price_usd,inv_variation,inv_accuracy,inv_shrink,avg_counting_days,sm_delivered,expected_skus,delivered_skus,late_sm,sku_issues,sold_items,wasted_items,avg_itemns_on_hand,f_count_availability,lost_gmv_auto86,lost_gmv_inv,avg_availability_stock_ob,percentage_odr,percentage_controllable_odr
6,2023-09-18,38,2023-09-18,38,ffe35195-a276-4537-8072-6875e100e579,Tier 1,2023-09-06,Gelati Artigianale,0015G00001be2FwQAI,Gelati - Pik n' Pak,Brazil,CR - Frozen Desserts,CR - Gelato,5103.0,Luiza Kravtchenko,NaN,NaN,NaN,NaN,4.87,36.0,7.392197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5275,7.51684,-4.1,0.933161,0.023661,5.067540640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [57]:
lista_df_2=[]
lista_semanas=[esta_semana, semana_pasada,dos_semanas, tres_semanas,cuatro_semanas, cinco_semanas, seis_semanas]
mydict={'this_week':esta_semana,'last_week':semana_pasada,'two_weeks':dos_semanas, 
              'three_weeks':tres_semanas,'four_weeks':cuatro_semanas, 'five_weeks':cinco_semanas, 
              'six_weeks':seis_semanas}

columnas=['created_week', 'isoweek','org_id','accountID',
       'tier','Account_Name_Jira__c','org_name','social_media_presence',
           #'Type', 'Renewal_Type__c','campaign_name',
       'Kitchen_Country__c','CSM',
         'orders', 'gmv_usd','f_count_availability','inv_accuracy',
         'inv_shrink' ,'avg_discount','percentage_odr',
       'percentage_controllable_odr']
unica_cuenta_2=unica_cuenta[columnas].rename(columns={'f_count_availability':'Inventory_Availability'}).reset_index(drop=True)

for i in lista_semanas:
    nuevo_df=unica_cuenta_2[unica_cuenta_2['isoweek']==i].reset_index(drop=True)
    nuevo_df_2=nuevo_df.set_index('accountID')
    if i==esta_semana:
        nuevo_df_3=nuevo_df_2.copy()
    else:
        sufijo=list(mydict.keys())[list(mydict.values()).index(i)]
        nuevo_df_3=nuevo_df_2.add_suffix('_'+sufijo)
    
    lista_df_2.append(nuevo_df_3)

    


In [58]:
semanas_cuenta=pd.concat(lista_df_2, axis=1).reset_index()

In [59]:
semanas_cuenta

,accountID,created_week,isoweek,org_id,tier,Account_Name_Jira__c,org_name,social_media_presence,Kitchen_Country__c,CSM,orders,gmv_usd,Inventory_Availability,inv_accuracy,inv_shrink,avg_discount,percentage_odr,percentage_controllable_odr,created_week_last_week,isoweek_last_week,org_id_last_week,tier_last_week,Account_Name_Jira__c_last_week,org_name_last_week,social_media_presence_last_week,Kitchen_Country__c_last_week,CSM_last_week,orders_last_week,gmv_usd_last_week,Inventory_Availability_last_week,inv_accuracy_last_week,inv_shrink_last_week,avg_discount_last_week,percentage_odr_last_week,percentage_controllable_odr_last_week,created_week_two_weeks,isoweek_two_weeks,org_id_two_weeks,tier_two_weeks,Account_Name_Jira__c_two_weeks,org_name_two_weeks,social_media_presence_two_weeks,Kitchen_Country__c_two_weeks,CSM_two_weeks,orders_two_weeks,gmv_usd_two_weeks,Inventory_Availability_two_weeks,inv_accuracy_two_weeks,inv_shrink_two_weeks,avg_discount_two_weeks,percentage_odr_two_weeks,percentage_controllable_odr_two_weeks,created_week_three_weeks,isoweek_three_weeks,org_id_three_weeks,tier_three_weeks,Account_Name_Jira__c_three_weeks,org_name_three_weeks,social_media_presence_three_weeks,Kitchen_Country__c_three_weeks,CSM_three_weeks,orders_three_weeks,gmv_usd_three_weeks,Inventory_Availability_three_weeks,inv_accuracy_three_weeks,inv_shrink_three_weeks,avg_discount_three_weeks,percentage_odr_three_weeks,percentage_controllable_odr_three_weeks,created_week_four_weeks,isoweek_four_weeks,org_id_four_weeks,tier_four_weeks,Account_Name_Jira__c_four_weeks,org_name_four_weeks,social_media_presence_four_weeks,Kitchen_Country__c_four_weeks,CSM_four_weeks,orders_four_weeks,gmv_usd_four_weeks,Inventory_Availability_four_weeks,inv_accuracy_four_weeks,inv_shrink_four_weeks,avg_discount_four_weeks,percentage_odr_four_weeks,percentage_controllable_odr_four_weeks,created_week_five_weeks,isoweek_five_weeks,org_id_five_weeks,tier_five_weeks,Account_Name_Jira__c_five_weeks,org_name_five_weeks,social_media_presence_five_weeks,Kitchen_Country__c_five_weeks,CSM_five_weeks,orders_five_weeks,gmv_usd_five_weeks,Inventory_Availability_five_weeks,inv_accuracy_five_weeks,inv_shrink_five_weeks,avg_discount_five_weeks,percentage_odr_five_weeks,percentage_controllable_odr_five_weeks,created_week_six_weeks,isoweek_six_weeks,org_id_six_weeks,tier_six_weeks,Account_Name_Jira__c_six_weeks,org_name_six_weeks,social_media_presence_six_weeks,Kitchen_Country__c_six_weeks,CSM_six_weeks,orders_six_weeks,gmv_usd_six_weeks,Inventory_Availability_six_weeks,inv_accuracy_six_weeks,inv_shrink_six_weeks,avg_discount_six_weeks,percentage_odr_six_weeks,percentage_controllable_odr_six_weeks
0,001f400001MFS1fAAH,2023-09-18,38,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,Lowko Sorvetes,58400.0,Brazil,Luiza Kravtchenko,NaN,NaN,NaN,0.955827,0.011187,NaN,NaN,NaN,2023-09-11,37,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,Lowko Sorvetes,58400.0,Brazil,Luiza Kravtchenko,59.0,905.934292,0.548959,0.936824,0.024002,0.130238,0.084746,0.000000,2023-09-04,36,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,Lowko Sorvetes,58400.0,Brazil,Luiza Kravtchenko,72.0,978.254620,0.693447,0.925037,0.053750,0.000000,0.083333,0.013889,2023-08-28,35,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,Lowko Sorvetes,58400.0,Brazil,Luiza Kravtchenko,12.0,166.204082,0.695588,0.926115,0.050995,0.000000,0.083333,0.083333,2023-08-21,34,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,Lowko Sorvetes,58400.0,Brazil,Luiza Kravtchenko,52.0,783.163265,0.629049,0.939355,0.016047,0.000000,0.038462,0.000000,2023-08-14,33,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,Lowko Sorvetes,58400.0,Brazil,Luiza Kravtchenko,73.0,1126.163265,0.656230,0.920191,0.029318,0.000000,0.000000,0.000000,2023-08-07,32,7f5836eb-9705-4854-b6ec-67f910428385,Tier 2,Lowko Sorvetes,Lowko Sorvetes,58400.0,Brazil,Luiza Kravtchenko,43.0,607.244898,0.621160,0.976879,0.002803,0.000000,0.0000

In [60]:
semanas_cuenta.columns

Index(['accountID', 'created_week', 'isoweek', 'org_id', 'tier',
       'Account_Name_Jira__c', 'org_name', 'social_media_presence',
       'Kitchen_Country__c', 'CSM',
       ...
       'Kitchen_Country__c_six_weeks', 'CSM_six_weeks', 'orders_six_weeks',
       'gmv_usd_six_weeks', 'Inventory_Availability_six_weeks',
       'inv_accuracy_six_weeks', 'inv_shrink_six_weeks',
       'avg_discount_six_weeks', 'percentage_odr_six_weeks',
       'percentage_controllable_odr_six_weeks'],
      dtype='object', length=120)

In [61]:
semanas_cuenta['accountID']

0      001f400001MFS1fAAH
1      0015G00001VcL4EQAV
2      0015G00001VfcFOQAZ
3      0015G00001a90ccQAA
4      001f400001Rk22DAAR
              ...        
290    001f400001OdutJAAR
291    0015G000024533yQAA
292    0016T00002yO5xsQAC
293    0015G000025Ch9DQAS
294    0016T000031ikmIQAQ
Name: accountID, Length: 295, dtype: object

In [62]:
semanas_cuenta_2=semanas_cuenta[['created_week', 'isoweek','org_id','accountID',
       'tier','Account_Name_Jira__c','org_name', 'Kitchen_Country__c','social_media_presence','CSM',
         
         
         
         'orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]]

In [63]:
semanas_cuenta_2

,created_week,isoweek,org_id,accountID,tier,Account_Name_Jira__c,org_name,Kitchen_Country__c,social_media_presence,CSM,orders,orders_last_week,orders_two_weeks,orders_three_weeks,orders_four_weeks,orders_five_weeks,orders_six_weeks,gmv_usd,gmv_usd_last_week,gmv_usd_two_weeks,gmv_usd_three_weeks,gmv_usd_four_weeks,gmv_usd_five_weeks,gmv_usd_six_weeks,Inventory_Availability,Inventory_Availability_last_week,Inventory_Availability_two_weeks,Inventory_Availability_three_weeks,Inventory_Availability_four_weeks,Inventory_Availability_five_weeks,Inventory_Availability_six_weeks,inv_accuracy,inv_accuracy_last_week,inv_accuracy_two_weeks,inv_accuracy_three_weeks,inv_accuracy_four_weeks,inv_accuracy_five_weeks,inv_accuracy_six_weeks,inv_shrink,inv_shrink_last_week,inv_shrink_two_weeks,inv_shrink_three_weeks,inv_shrink_four_weeks,inv_shrink_five_weeks,inv_shrink_six_weeks,avg_discount,avg_discount_last_week,avg_discount_two_weeks,avg_discount_three_weeks,avg_discount_four_weeks,avg_discount_five_weeks,avg_discount_six_weeks,percentage_odr,percentage_odr_last_week,percentage_odr_two_weeks,percentage_odr_three_weeks,percentage_odr_four_weeks,percentage_odr_five_weeks,percentage_odr_six_weeks,percentage_controllable_odr,percentage_controllable_odr_last_week,percentage_controllable_odr_two_weeks,percentage_controllable_odr_three_weeks,percentage_controllable_odr_four_weeks,percentage_controllable_odr_five_weeks,percentage_controllable_odr_six_weeks
0,2023-09-18,38,7f5836eb-9705-4854-b6ec-67f910428385,001f400001MFS1fAAH,Tier 2,Lowko Sorvetes,Lowko Sorvetes,Brazil,58400.0,Luiza Kravtchenko,NaN,59.0,72.0,12.0,52.0,73.0,43.0,NaN,905.934292,978.254620,166.204082,783.163265,1126.163265,607.244898,NaN,0.548959,0.693447,0.695588,0.629049,0.656230,0.621160,0.955827,0.936824,0.925037,0.926115,0.939355,0.920191,0.976879,0.011187,0.024002,0.053750,0.050995,0.016047,0.029318,0.002803,NaN,0.130238,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.084746,0.083333,0.083333,0.038462,0.000000,0.000000,NaN,0.000000,0.013889,0.083333,0.000000,0.000000,0.000000
1,2023-09-18,38,dd7d79c0-6d5e-4ee3-8f3d-d19145e028be,0015G00001VcL4EQAV,Tier 2,Brigadeiro com VC,Brigadeiro Com Vc - Pik n' Pak,Brazil,14800.0,Luiza Kravtchenko,NaN,51.0,31.0,41.0,54.0,55.0,42.0,NaN,413.008214,275.359343,345.247152,458.202041,528.030612,340.312245,NaN,0.425275,0.608395,0.586697,0.551196,0.392582,0.446838,1.000000,1.000000,0.954443,0.926325,0.973345,0.945443,0.895429,0.000000,0.000000,0.014272,0.022766,0.002079,0.026163,0.030495,NaN,0.233087,0.205690,0.147198,0.142879,0.160581,0.162041,NaN,0.117647,0.096774,0.073171,0.037037,0.072727,0.142857,NaN,0.058824,0.000000,0.024390,0.018519,0.036364,0.071429
2,2023-09-18,38,576412a2-e855-487a-a92b-5402b2cd95b4,0015G00001VfcFOQAZ,Tier 3,Good Pick,Good Pick - Pik n' Pak,Brazil,5697.0,Rhayla Freitas Gualberto,NaN,2.0,1.0,4.0,NaN,NaN,2.0,NaN,43.572895,63.983573,166.612245,NaN,NaN,74.367347,NaN,0.900000,1.000000,0.885204,0.571429,0.591837,0.561224,1.000000,1.000000,0.909524,0.642857,1.000000,0.809620,0.809620,0.000000,0.000000,0.090476,0.357143,0.000000,0.190380,0.190380,NaN,0.175000,0.100000,0.550000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000
3,2023-09-18,38,c0b76b32-019e-49b5-9c27-b1cc2d042770,0015G00001a90ccQAA,Tier 3,Mezcal Amores,Mezcal Amores,Mexico,119000.0,Diana Echeverry,NaN,8.0,NaN,4.0,1.0,2.0,4.0,NaN,338.686987,NaN,132.998828,33.914016,85.090106,349.091873,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.984694,0.979592,1.000000,0.997599,1.000000,1.000000,0.000000,0.015306,0.020408,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
4,2023-09-18,38,1ee553b1-0874-4bc4-a5ce-407a2b943eef,001f400001Rk22DAAR,Tier 1,Pudim Box,Pudim Box,Brazil,4047.0,Luiza Kravtchenko,NaN,408.0,512.0,400.0,585.0,608.0,976.0,NaN,4717.687885,66

In [64]:
semanas_cuenta_2 [['orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
            
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]]=semanas_cuenta_2 [[ 'orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]].fillna(0)

In [65]:
semanas_num_cuenta=semanas_cuenta_2 [['orders','orders_last_week','orders_two_weeks','orders_three_weeks','orders_four_weeks','orders_five_weeks','orders_six_weeks',
         'gmv_usd', 'gmv_usd_last_week','gmv_usd_two_weeks','gmv_usd_three_weeks','gmv_usd_four_weeks','gmv_usd_five_weeks','gmv_usd_six_weeks',
         'Inventory_Availability','Inventory_Availability_last_week','Inventory_Availability_two_weeks','Inventory_Availability_three_weeks','Inventory_Availability_four_weeks','Inventory_Availability_five_weeks','Inventory_Availability_six_weeks',
         'inv_accuracy','inv_accuracy_last_week','inv_accuracy_two_weeks','inv_accuracy_three_weeks','inv_accuracy_four_weeks','inv_accuracy_five_weeks','inv_accuracy_six_weeks',
         'inv_shrink' , 'inv_shrink_last_week','inv_shrink_two_weeks', 'inv_shrink_three_weeks','inv_shrink_four_weeks','inv_shrink_five_weeks','inv_shrink_six_weeks',
         'avg_discount','avg_discount_last_week','avg_discount_two_weeks','avg_discount_three_weeks','avg_discount_four_weeks','avg_discount_five_weeks', 'avg_discount_six_weeks',
         'percentage_odr','percentage_odr_last_week','percentage_odr_two_weeks','percentage_odr_three_weeks','percentage_odr_four_weeks','percentage_odr_five_weeks','percentage_odr_six_weeks',
         'percentage_controllable_odr','percentage_controllable_odr_last_week','percentage_controllable_odr_two_weeks','percentage_controllable_odr_three_weeks','percentage_controllable_odr_four_weeks','percentage_controllable_odr_five_weeks','percentage_controllable_odr_six_weeks'
        ]]

In [66]:
for i in range(len(semanas_num_cuenta.columns)):
    #print(semanas_num.columns[i])
    if i==0:
        pass
    else:
        nuevo_nombre='delta_'+semanas_num_cuenta.columns[i]+'-'+semanas_num_cuenta.columns[i-1]
        print(nuevo_nombre)
        
        semanas_cuenta_2[nuevo_nombre]=((semanas_num_cuenta[semanas_num_cuenta.columns[i-1]]/semanas_num_cuenta[semanas_num_cuenta.columns[i]])-1).replace(np.inf,0).replace(-np.inf,0).fillna(0)

delta_orders_last_week-orders
delta_orders_two_weeks-orders_last_week
delta_orders_three_weeks-orders_two_weeks
delta_orders_four_weeks-orders_three_weeks
delta_orders_five_weeks-orders_four_weeks
delta_orders_six_weeks-orders_five_weeks
delta_gmv_usd-orders_six_weeks
delta_gmv_usd_last_week-gmv_usd
delta_gmv_usd_two_weeks-gmv_usd_last_week
delta_gmv_usd_three_weeks-gmv_usd_two_weeks
delta_gmv_usd_four_weeks-gmv_usd_three_weeks
delta_gmv_usd_five_weeks-gmv_usd_four_weeks
delta_gmv_usd_six_weeks-gmv_usd_five_weeks
delta_Inventory_Availability-gmv_usd_six_weeks
delta_Inventory_Availability_last_week-Inventory_Availability
delta_Inventory_Availability_two_weeks-Inventory_Availability_last_week
delta_Inventory_Availability_three_weeks-Inventory_Availability_two_weeks
delta_Inventory_Availability_four_weeks-Inventory_Availability_three_weeks
delta_Inventory_Availability_five_weeks-Inventory_Availability_four_weeks
delta_Inventory_Availability_six_weeks-Inventory_Availability_five_weeks
delt

In [67]:
semanas_cuenta_exp=semanas_cuenta_2[['created_week', 'isoweek','org_id','accountID',
       'tier','Account_Name_Jira__c','Kitchen_Country__c','social_media_presence','CSM',
         
         
         
         'orders','orders_last_week','delta_orders_last_week-orders','orders_two_weeks','delta_orders_two_weeks-orders_last_week','orders_three_weeks','delta_orders_three_weeks-orders_two_weeks',
        'orders_four_weeks','delta_orders_four_weeks-orders_three_weeks','orders_five_weeks','delta_orders_five_weeks-orders_four_weeks','orders_six_weeks','delta_orders_six_weeks-orders_five_weeks',
                       
                       
         'gmv_usd', 'gmv_usd_last_week','delta_gmv_usd_last_week-gmv_usd','gmv_usd_two_weeks','delta_gmv_usd_two_weeks-gmv_usd_last_week','gmv_usd_three_weeks','delta_gmv_usd_three_weeks-gmv_usd_two_weeks',
        'gmv_usd_four_weeks', 'delta_gmv_usd_four_weeks-gmv_usd_three_weeks','gmv_usd_five_weeks','delta_gmv_usd_five_weeks-gmv_usd_four_weeks','gmv_usd_six_weeks','delta_gmv_usd_six_weeks-gmv_usd_five_weeks',
                       
                       
                       
         'Inventory_Availability','Inventory_Availability_last_week','delta_Inventory_Availability_last_week-Inventory_Availability','Inventory_Availability_two_weeks','delta_Inventory_Availability_two_weeks-Inventory_Availability_last_week',
        'Inventory_Availability_three_weeks','delta_Inventory_Availability_three_weeks-Inventory_Availability_two_weeks','Inventory_Availability_four_weeks','delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks',
        'Inventory_Availability_five_weeks','delta_Inventory_Availability_five_weeks-Inventory_Availability_four_weeks','Inventory_Availability_six_weeks','delta_Inventory_Availability_six_weeks-Inventory_Availability_five_weeks',
         
                       
                       
        'inv_accuracy','inv_accuracy_last_week','delta_inv_accuracy_last_week-inv_accuracy','inv_accuracy_two_weeks','delta_inv_accuracy_two_weeks-inv_accuracy_last_week','inv_accuracy_three_weeks','delta_inv_accuracy_three_weeks-inv_accuracy_two_weeks',
        'inv_accuracy_four_weeks','delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks','inv_accuracy_five_weeks','delta_inv_accuracy_five_weeks-inv_accuracy_four_weeks','inv_accuracy_six_weeks','delta_inv_accuracy_six_weeks-inv_accuracy_five_weeks',
                       
                       
                       
         'inv_shrink' , 'inv_shrink_last_week','delta_inv_shrink_last_week-inv_shrink','inv_shrink_two_weeks','inv_shrink_three_weeks','delta_inv_shrink_three_weeks-inv_shrink_two_weeks',
        'inv_shrink_four_weeks','delta_Inventory_Availability_four_weeks-Inventory_Availability_three_weeks','inv_shrink_five_weeks','delta_inv_shrink_five_weeks-inv_shrink_four_weeks','inv_shrink_six_weeks','delta_inv_shrink_six_weeks-inv_shrink_five_weeks',
                       
                       
                       
         'avg_discount','avg_discount_last_week','delta_avg_discount_last_week-avg_discount','avg_discount_two_weeks','delta_avg_discount_two_weeks-avg_discount_last_week','avg_discount_three_weeks','delta_avg_discount_three_weeks-avg_discount_two_weeks',
        'avg_discount_four_weeks','delta_avg_discount_four_weeks-avg_discount_three_weeks','avg_discount_five_weeks', 'delta_avg_discount_five_weeks-avg_discount_four_weeks','avg_discount_six_weeks','delta_avg_discount_six_weeks-avg_discount_five_weeks',
                       
                       
         'percentage_odr','percentage_odr_last_week','delta_percentage_odr_last_week-percentage_odr','percentage_odr_two_weeks', 'delta_percentage_odr_two_weeks-percentage_odr_last_week',
           'percentage_odr_three_weeks','delta_percentage_odr_three_weeks-percentage_odr_two_weeks','percentage_odr_four_weeks','delta_percentage_odr_four_weeks-percentage_odr_three_weeks',
        'percentage_odr_five_weeks','delta_percentage_odr_five_weeks-percentage_odr_four_weeks','percentage_odr_six_weeks','delta_percentage_controllable_odr-percentage_odr_six_weeks',
                       
         'percentage_controllable_odr','percentage_controllable_odr_last_week','delta_percentage_controllable_odr_last_week-percentage_controllable_odr',
        'percentage_controllable_odr_two_weeks','delta_percentage_controllable_odr_last_week-percentage_controllable_odr','percentage_controllable_odr_three_weeks','delta_percentage_controllable_odr_three_weeks-percentage_controllable_odr_two_weeks',
        'percentage_controllable_odr_four_weeks','delta_percentage_controllable_odr_four_weeks-percentage_controllable_odr_three_weeks','percentage_controllable_odr_five_weeks','delta_percentage_controllable_odr_five_weeks-percentage_controllable_odr_four_weeks',
            'percentage_controllable_odr_six_weeks','delta_percentage_controllable_odr_six_weeks-percentage_controllable_odr_five_weeks'
        ]]

In [68]:
semanas_cuenta_exp

,created_week,isoweek,org_id,accountID,tier,Account_Name_Jira__c,Kitchen_Country__c,social_media_presence,CSM,orders,orders_last_week,delta_orders_last_week-orders,orders_two_weeks,delta_orders_two_weeks-orders_last_week,orders_three_weeks,delta_orders_three_weeks-orders_two_weeks,orders_four_weeks,delta_orders_four_weeks-orders_three_weeks,orders_five_weeks,delta_orders_five_weeks-orders_four_weeks,orders_six_weeks,delta_orders_six_weeks-orders_five_weeks,gmv_usd,gmv_usd_last_week,delta_gmv_usd_last_week-gmv_usd,gmv_usd_two_weeks,delta_gmv_usd_two_weeks-gmv_usd_last_week,gmv_usd_three_weeks,delta_gmv_usd_three_weeks-gmv_usd_two_weeks,gmv_usd_four_weeks,delta_gmv_usd_four_weeks-gmv_usd_three_weeks,gmv_usd_five_weeks,delta_gmv_usd_five_weeks-gmv_usd_four_weeks,gmv_usd_six_weeks,delta_gmv_usd_six_weeks-gmv_usd_five_weeks,Inventory_Availability,Inventory_Availability_last_week,delta_Inventory_Availability_last_week-Inventory_Availability,Inventory_Availability_two_weeks,delta_Inventory_Availability_two_weeks-Inventory_Availability_last_week,Inventory_Availability_three_weeks,delta_Inventory_Availability_three_weeks-Inventory_Availability_two_weeks,Inventory_Availability_four_weeks,delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks,Inventory_Availability_five_weeks,delta_Inventory_Availability_five_weeks-Inventory_Availability_four_weeks,Inventory_Availability_six_weeks,delta_Inventory_Availability_six_weeks-Inventory_Availability_five_weeks,inv_accuracy,inv_accuracy_last_week,delta_inv_accuracy_last_week-inv_accuracy,inv_accuracy_two_weeks,delta_inv_accuracy_two_weeks-inv_accuracy_last_week,inv_accuracy_three_weeks,delta_inv_accuracy_three_weeks-inv_accuracy_two_weeks,inv_accuracy_four_weeks,delta_inv_accuracy_four_weeks-inv_accuracy_three_weeks,inv_accuracy_five_weeks,delta_inv_accuracy_five_weeks-inv_accuracy_four_weeks,inv_accuracy_six_weeks,delta_inv_accuracy_six_weeks-inv_accuracy_five_weeks,inv_shrink,inv_shrink_last_week,delta_inv_shrink_last_week-inv_shrink,inv_shrink_two_weeks,inv_shrink_three_weeks,delta_inv_shrink_three_weeks-inv_shrink_two_weeks,inv_shrink_four_weeks,delta_Inventory_Availability_four_weeks-Inventory_Availability_three_weeks,inv_shrink_five_weeks,delta_inv_shrink_five_weeks-inv_shrink_four_weeks,inv_shrink_six_weeks,delta_inv_shrink_six_weeks-inv_shrink_five_weeks,avg_discount,avg_discount_last_week,delta_avg_discount_last_week-avg_discount,avg_discount_two_weeks,delta_avg_discount_two_weeks-avg_discount_last_week,avg_discount_three_weeks,delta_avg_discount_three_weeks-avg_discount_two_weeks,avg_discount_four_weeks,delta_avg_discount_four_weeks-avg_discount_three_weeks,avg_discount_five_weeks,delta_avg_discount_five_weeks-avg_discount_four_weeks,avg_discount_six_weeks,delta_avg_discount_six_weeks-avg_discount_five_weeks,percentage_odr,percentage_odr_last_week,delta_percentage_odr_last_week-percentage_odr,percentage_odr_two_weeks,delta_percentage_odr_two_weeks-percentage_odr_last_week,percentage_odr_three_weeks,delta_percentage_odr_three_weeks-percentage_odr_two_weeks,percentage_odr_four_weeks,delta_percentage_odr_four_weeks-percentage_odr_three_weeks,percentage_odr_five_weeks,delta_percentage_odr_five_weeks-percentage_odr_four_weeks,percentage_odr_six_weeks,delta_percentage_controllable_odr-percentage_odr_six_weeks,percentage_controllable_odr,percentage_controllable_odr_last_week,delta_percentage_controllable_odr_last_week-percentage_controllable_odr,percentage_controllable_odr_two_weeks,delta_percentage_controllable_odr_last_week-percentage_controllable_odr,percentage_controllable_odr_three_weeks,delta_percentage_controllable_odr_three_weeks-percentage_controllable_odr_two_weeks,percentage_controllable_odr_four_weeks,delta_percentage_controllable_odr_four_weeks-percentage_controllable_odr_three_weeks,percentage_controllable_odr_five_weeks,delta_percentage_controllable_odr_five_weeks-percentage_controllable_odr_four_weeks,percentage_controllable_odr_six_weeks,delta_percentage_controllable_odr_six_w

In [69]:
# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1zir2UeDnoOC12k9pfu913s4BbkX2Qs2MTbHjlta0enM')

# select sheet

ws = ss.worksheet('title','Account data')

print('Clearing Range ...')
ws.clear(start='A', end='DT')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(semanas_cuenta_exp, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [70]:
#saulo

In [71]:
big_query_saulo="""


SELECT

date(DATE_TRUNC(fecha, week(MONDAY))) as created_week, (EXTRACT(ISOWEEK FROM FECHA)) as isoweek,

org_id,tier,date(Closed_Won_Date__c) as close_won_date,
date(Move_In_Date__c) as move_in_date, date(Start_Date__c) as start_date,
date(Churn_Date__c) as churned_date, date(last_task_date) as last_task_date,
Account_Name_Jira__c,Opportunity_ID_18__c,Facility_Name_fixed as facility_name,
Kitchen_Country__c,
MRR_local, storage_ft3,category, subcategory, social_media_presence, CSM, Primary_Churn_Reason__c,
CR_Opp_OB_Status__c,

sum(orders) as orders,
sum(gmv_local) as gmv_local,
sum(gmv_usd) as gmv_usd,
sum(cancelled_gmv_usd) as cancelled_gmv_usd,
sum(cancelled_gmv_local) as cancelled_gmv_local,

avg(f_count_availability) as f_count_availability,
avg(inv_variation) as inv_variation,
avg(inv_accuracy) as inv_accuracy,
avg(inv_shrink) as inv_shrink,
sum(items_wasted) as wasted_items,
avg( avg_discount) as avg_discount,

sum(odr_orders) as odr_orders,
sum(Controllable_ord_orders) as Controllable_ord_orders,
sum(Non_controllable_ord_orders) as Non_controllable_ord_orders,
sum(issue_orders) as issue_orders,
sum(cancelled_orders) as cancelled_orders,
sum(odr_orders)/sum(orders) as percentage_odr,
sum(Controllable_ord_orders)/sum(orders) as percentage_controllable_odr






FROM `css-operations.latam_cr_growth.tabla_unica`

where date(fecha)>='2023-01-01'
and Move_In_Date__c is not null
and Churn_Date__c is null
and Kitchen_Country__c='Brazil'
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
order by 3,30 desc

"""

In [72]:
saulo=pd.read_gbq(query=big_query_saulo, credentials=credentials,
                 dialect='standard')

In [73]:
saulo

,created_week,isoweek,org_id,tier,close_won_date,move_in_date,start_date,churned_date,last_task_date,Account_Name_Jira__c,Opportunity_ID_18__c,facility_name,Kitchen_Country__c,MRR_local,storage_ft3,category,subcategory,social_media_presence,CSM,Primary_Churn_Reason__c,CR_Opp_OB_Status__c,orders,gmv_local,gmv_usd,cancelled_gmv_usd,cancelled_gmv_local,f_count_availability,inv_variation,inv_accuracy,inv_shrink,wasted_items,avg_discount,odr_orders,Controllable_ord_orders,Non_controllable_ord_orders,issue_orders,cancelled_orders,percentage_odr,percentage_controllable_odr
0,2023-09-04,36,02252223-e0cf-4582-bad0-c54ee452163e,Tier 3,2023-05-17,2023-06-27,2023-06-26,NaT,NaT,Mahalo Bolos,0066T00001C9b0ZQAR,SAO - Guararapes (Brooklin),Brazil,900.0,8.0,CR - Baked Goods,CR - Cakes,1066.0,Rhayla Freitas Gualberto,None,g - Launched,4.0,125.40,25.749487,0.0,0.0,0.414966,1.428571e+00,0.967532,0.032468,0.0,0.282695,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-08-14,33,02252223-e0cf-4582-bad0-c54ee452163e,Tier 3,2023-05-17,2023-06-27,2023-06-26,NaT,NaT,Mahalo Bolos,0066T00001C9b0ZQAR,SAO - Guararapes (Brooklin),Brazil,900.0,8.0,CR - Baked Goods,CR - Cakes,1066.0,Rhayla Freitas Gualberto,None,g - Launched,4.0,109.50,22.346939,0.0,0.0,0.551020,2.857143e-01,0.968254,0.015873,0.0,0.274171,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-08-07,32,02252223-e0cf-4582-bad0-c54ee452163e,Tier 3,2023-05-17,2023-06-27,2023-06-26,NaT,NaT,Mahalo Bolos,0066T00001C9b0ZQAR,SAO - Guararapes (Brooklin),Brazil,900.0,8.0,CR - Baked Goods,CR - Cakes,1066.0,Rhayla Freitas Gualberto,None,g - Launched,5.0,216.10,44.102041,0.0,0.0,0.630952,-3.965082e-17,0.932540,0.015873,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-08-28,35,02252223-e0cf-4582-bad0-c54ee452163e,Tier 3,2023-05-17,2023-06-27,2023-06-26,NaT,NaT,Mahalo Bolos,0066T00001C9b0ZQAR,SAO - Guararapes (Brooklin),Brazil,900.0,8.0,CR - Baked Goods,CR - Cakes,1066.0,Rhayla Freitas Gualberto,None,g - Launched,NaN,NaN,NaN,NaN,NaN,0.357143,-2.857143e-01,0.947941,0.012987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-09-11,37,02252223-e0cf-4582-bad0-c54ee452163e,Tier 3,2023-05-17,2023-06-27,2023-06-26,NaT,NaT,Mahalo Bolos,0066T00001C9b0ZQAR,SAO - Guararapes (Brooklin),Brazil,900.0,8.0,CR - Baked Goods,CR - Cakes,1066.0,Rhayla Freitas Gualberto,None,g - Launched,4.0,161.21,33.102669,0.0,0.0,0.500000,0.000000e+00,1.000000,0.000000,0.0,0.185552,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15166,2023-01-09,2,ffe35195-a276-4537-8072-6875e100e579.,Tier 1,2023-02-28,2023-03-10,2023-03-10,NaT,2023-06-07,PIZZA AL TAGLIO LTDA (Gelati),0066T00001AuXPbQAN,SAO - Guararapes (Brooklin),Brazil,1150.0,20.0,CR - Frozen Desserts,CR - Ice Cream,4411.0,Luiza Kravtchenko,None,g - Launched,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15167,2023-04-03,14,ffe35195-a276-4537-8072-6875e100e579.,Tier 1,2023-02-28,2023-03-10,2023-03-10,NaT,2023-06-07,PIZZA AL TAGLIO LTDA (Gelati),0066T00001AuXPbQAN,SAO - Guararapes (Brooklin),Brazil,1150.0,20.0,CR - Frozen Desserts,CR - Ice Cream,4411.0,Luiza Kravtchenko,None,g - Launched,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15168,2023-02-27,9,ffe35195-a276-4537-8072-6875e100e579.,Tier 1,2023-02-28,2023-03-10,2023-03-10,NaT,2023-06-07,PIZZA AL TAGLIO LTDA (Gelati),0066T00001AuXPbQAN,SAO - Guararapes (Brooklin),Brazil,1150.0,20.0,CR - Frozen Desserts,CR - Ice Cream,4411.0,Luiza Kravtchenko,None,g - Launched,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15169,2023-05-22,21,ffe35195-a276-4537-8072-6875e100e579.,Tier 1,2023-02-28,2023-03-10,2023-03-10,NaT,2023-06-07,PIZZA AL TAGLIO LTDA (Gelati),0066T00001AuXPbQAN,SAO - Guararapes (Brooklin),Brazil,1150.0,20.0,CR - Frozen Desserts,CR - Ice Cream,4411.0,Luiza Kravtchenko,None,g - Launched,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1D32Mn2CyfbO6HeZ3ANFGr8-eRYawcuguFEp7xIgjQhI')

# select sheet

ws = ss.worksheet('title','wow_data_2')

print('Clearing Range ...')
ws.clear(start='A', end='DT')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(saulo, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!
